In [1]:
!pip install mlflow torch torchinfo boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 109.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 122.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.7/629.7 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.4/602.4 kB 20.9 MB/s eta 0:00:00


In [2]:
import os
import mlflow
import torch

assert torch.version.__version__ >= "1.0.0"

import torch.nn as nn
from torchinfo import summary
import torch.nn.functional as F
import torch.optim as optim

os.environ["MLFLOW_S3_ENDPOINT_URL"] = (
    "https://jku-project-mlflow.nyc3.digitaloceanspaces.com"
)
os.environ["AWS_ACCESS_KEY_ID"] = "DO00JG8HAFM7XN7EYZEU"
os.environ["AWS_SECRET_ACCESS_KEY"] = "mBHWu6861PCNFd1/rifx8zZ6brWwemFBElDFX3tzPY8"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mlflow.set_tracking_uri("http://mlflow.default.svc.cluster.local:5000")

# mlflow.create_experiment("c2", artifact_location="s3://jku-project-mlflow/mlflow")
mlflow.set_experiment("c2")


class XOR(nn.Module):
    def __init__(self):
        super(XOR, self).__init__()
        self.fc1 = nn.Linear(2, 4, True)
        self.fc2 = nn.Linear(4, 1, True)

    def forward(self, x):
        h = F.tanh(self.fc1(x))
        y = F.sigmoid(self.fc2(h))
        return y


def train(model, loss_fn, optimizer):
    model = model.to(device)

    X = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], device=device).float()
    y = torch.tensor([[0], [1], [1], [0]], device=device).float()

    model.train()

    pred = model(X)
    _loss = loss_fn(pred, y)
    _loss.backward()

    optimizer.step()
    optimizer.zero_grad()

    return _loss


if __name__ == "__main__":
    epochs = 5
    loss_fn = nn.BCELoss()
    model = XOR().to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.1)

    with mlflow.start_run() as run:
        mlflow.doctor()
        # artifact_uri = mlflow.get_artifact_uri()
        # print(f"Artifact uri: {artifact_uri}")
        # print(mlflow.get_artifact_uri())
        params = {
            "epochs": epochs,
            "learning_rate": 1e-3,
            "loss_function": loss_fn.__class__.__name__,
            "optimizer": "SGD",
        }
        mlflow.log_params(params)

        with open("model_summary.txt", "w") as f:
            f.write(str(summary(model)))
            mlflow.log_artifact("model_summary.txt")

        for epoch in range(epochs):
            print(f"Epoch {epoch+1}\n-------------------------------")
            loss = train(model, loss_fn, optimizer)
            mlflow.log_metric(key="train_loss", value=loss, step=epoch)
            if epoch % 100 == 0:
                print(
                    "Epoch: {0}, Loss: {1}, ".format(
                        epoch, loss.to("cpu").detach().numpy()
                    )
                )

        mlflow.pytorch.log_model(
            model, registered_model_name="xor", artifact_path="xor-model"
        )

System information: Linux #1 SMP PREEMPT_DYNAMIC Debian 6.1.115-1 (2024-11-01)
Python version: 3.11.10
MLflow version: 2.19.0
MLflow module location: /opt/conda/lib/python3.11/site-packages/mlflow/__init__.py
Tracking URI: http://mlflow.default.svc.cluster.local:5000
Registry URI: http://mlflow.default.svc.cluster.local:5000
Active experiment ID: 4
Active run ID: 91da2897c4d647a58efb979c835ff769
Active run artifact URI: s3://jku-project-mlflow/mlflow/91da2897c4d647a58efb979c835ff769/artifacts
MLflow environment variables: 
  MLFLOW_EXPERIMENT_ID: 4
  MLFLOW_S3_ENDPOINT_URL: https://jku-project-mlflow.nyc3.digitaloceanspaces.com
  MLFLOW_TRACKING_URI: http://mlflow.default.svc.cluster.local:5000
MLflow dependencies: 
  Flask: 3.1.0
  Jinja2: 3.1.4
  alembic: 1.14.0
  boto3: 1.35.90
  botocore: 1.35.90
  docker: 7.1.0
  google-cloud-storage: 2.18.2
  graphene: 3.4.3
  gunicorn: 23.0.0
  kubernetes: 30.1.0
  markdown: 3.7
  matplotlib: 3.8.4
  mlflow-skinny: 2.19.0
  numpy: 1.24.4
  panda

2024/12/30 14:59:24 WARNING mlflow.utils.requirements_utils: Found torch version (2.3.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.3.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


Epoch 1
-------------------------------
Epoch: 0, Loss: 0.7207471132278442, 
Epoch 2
-------------------------------
Epoch 3
-------------------------------
Epoch 4
-------------------------------
Epoch 5
-------------------------------


2024/12/30 14:59:34 WARNING mlflow.utils.requirements_utils: Found torch version (2.3.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.3.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/12/30 14:59:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'xor' already exists. Creating a new version of this model...
2024/12/30 14:59:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: xor, version 3


🏃 View run sneaky-doe-620 at: http://mlflow.default.svc.cluster.local:5000/#/experiments/4/runs/91da2897c4d647a58efb979c835ff769
🧪 View experiment at: http://mlflow.default.svc.cluster.local:5000/#/experiments/4


Created version '3' of model 'xor'.
